# Lab | Data Cleaning Bonus Challenge

In [1]:
#importing the required libraries
import pandas as pd

In [2]:
iris = pd.read_csv('iris-data.csv')

In [3]:
#We have a first look at the data
iris.head()

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
iris.shape

(150, 5)

In [5]:
iris.describe()

,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm
count,150.000000,150.000000,150.000000,145.000000
mean,5.644627,3.054667,3.758667,1.236552
std,1.312781,0.433123,1.764420,0.755058
min,0.055000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.400000
50%,5.700000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


First thing we observe is that 'petal_width_cm' is missing 5 values
'sepal_length_cm' might have some outliers close to the min value

In [6]:
iris.hist(bins=20).show()

AttributeError: 'numpy.ndarray' object has no attribute 'show'

We can confirm the outlier in 'sepal_length_cm
In both petals length and width there are some values close to the min values which might have something to say.

In [ ]:
#Checking if there are null values
iris.isnull().sum()

In [ ]:
#Are the types correct? Yes, they are.
iris.dtypes

In [ ]:
iris.petal_width_cm.unique()

In [ ]:
#Let's identify where are the NaN values
missing_values = iris[iris.isnull().any(axis=1)]
missing_values

In [ ]:
#All the NaN values share the same class, let's see how many different classes there are and then identify the average attributes
#of iris-setosa
iris['class'].value_counts()

In [ ]:
#By checking the unique classes, we identify two mistakes, we could combine both 'Iris-setosa' and 'Iris-setossa' and both
#'versicolor' to finally have three unique classes. We will do this before finding out what to do with NaNs.
iris.replace(to_replace='Iris-setossa',value='Iris-setosa',inplace=True)

In [ ]:
iris.replace(to_replace='versicolor',value='Iris-versicolor',inplace=True)

In [ ]:
#Let's check again
iris['class'].value_counts()

In [ ]:
#We seemed to have fixed the problem of the classes, now let's go back to the NaN values
missing_values = iris[iris.isnull().any(axis=1)]
missing_values

In [ ]:
#Let's try to understand common properties among classes by grouping by them.
iris_class = iris.groupby('class')
iris_class.hist()

In [ ]:
iris.groupby('class').mean()

In [ ]:
iris_class.corr()

In [ ]:
iris_class.describe()

In [ ]:
iris_setosa = iris[iris['class'] == 'Iris-setosa']

In [ ]:
iris_setosa = iris_setosa.dropna()

In [ ]:
iris_setosa.shape

In [ ]:
import seaborn as sns
sns.regplot(x='petal_length_cm',y='petal_width_cm', data=iris_setosa)

In [ ]:
import statsmodels.api as sm
X = sm.add_constant(iris_setosa['petal_length_cm'])
model = sm.OLS(iris_setosa['petal_width_cm'],X)
results = model.fit()
print(results.summary())

In [ ]:
constant = -0.0433
petal_length = 0.2017

Things we know so far in order to take a decision with the NaNs:
- I would substitute the values of NaN because it feels that those are missing for a reason (they are in the same range 7-11 so maybe something happened the day this data was collected that petal_width_cm could not be measured.
- The mean of petal_width_cm in iris-setosa is 0.251111 and the mode is 0.2 which indicates that it is likely that the NaN value is around that area. But the coefficient of the linear regression compared to the length is 0.2017, so we will apply the formula to fill na.

In [ ]:
for i in range(0, 5):
    fill_value= constant + missing_values.petal_length_cm.iloc[i] * petal_length
    missing_values.petal_width_cm.iloc[i] = round(fill_value,2)
missing_values

In [ ]:
iris = iris.fillna(missing_values)

In [8]:
iris.to_csv('iris-data-clean.csv')